In [0]:
# !pip install --upgrade tables
# !pip install eli5
# !pip install xgboost
# !pip install hyperopt

In [0]:
import pandas as pd
import numpy as np


import xgboost as xgb

from sklearn.metrics import median_absolute_error as mae
from sklearn.model_selection import cross_val_score

from hyperopt import hp, fmin, tpe, STATUS_OK

import eli5
from eli5.sklearn import PermutationImportance

In [9]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_car'

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_car


In [10]:
ls

data/                     day3_simple_model.ipynb  LICENSE
day2_visualisation.ipynb  day4.ipynb               README.md


In [11]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

##Feature Engineering

In [0]:
SUFFIX_CAT = '__cat'

for feat in df.columns:
  if isinstance(df[feat][0],list): continue

  factorized_feats = df[feat].factorize()[0]

  if SUFFIX_CAT in feat:
    df[feat] = factorized_feats
  else: 
    df[feat + SUFFIX_CAT] = factorized_feats


In [0]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x : -1 if str(x) == 'None' else int(x))
df['param_moc'] = df['param_moc'].map(lambda x : -1 if str(x) =='None' else int(x.split('KM')[0].replace(' ','')))
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x : -1 if str(x) =='None' else int(x.split('cm')[0].replace(' ','')))

In [31]:
df['price_currency'].value_counts()

PLN    106290
EUR       204
Name: price_currency, dtype: int64

In [32]:
df = df[df['price_currency'] != 'EUR']
df.shape

(106290, 309)

In [0]:
def run_model(model, feat):
  X = df[feat].values
  y = df['price_value'].values
  
  scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)


In [0]:
feats = ['param_napęd__cat','param_rok-produkcji','param_stan__cat','param_skrzynia-biegów__cat','param_faktura-vat__cat',
'param_moc','param_marka-pojazdu__cat','param_typ__cat','feature_kamera-cofania__cat','param_pojemność-skokowa',
'seller_name__cat','param_kod-silnika__cat','param_model-pojazdu__cat','feature_wspomaganie-kierownicy__cat','param_wersja__cat',
'feature_czujniki-parkowania-przednie__cat','feature_asystent-pasa-ruchu__cat','feature_regulowane-zawieszenie__cat',
'feature_system-start-stop__cat','feature_światła-led__cat']

xgb_params = {
    'max_depth' : 5,
    'n_estimators' : 50,
    'learning_rate' : 0.1,
    'seed' : 0
}

In [35]:
run_model(xgb.XGBRegressor(**xgb_params),feats)


[09:36:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:36:12] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:36:16] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9461.390517191321, 83.66553795695546)

##Hyperopt

In [0]:
def obj_func(params):
  print('Training with params: ')
  print(params)

  mean_mae, score_std = run_model(xgb.XGBRegressor(**params),feats)

  return {'loss': np.abs(mean_mae), 'status':STATUS_OK}

In [38]:
##space

xgb_reg_params = {
    'learning_rate' :hp.choice('learning_rate', np.arange(0.05, 0.31, 0.05)) ,
    'max_depth' : hp.choice('max_depth', np.arange(5, 16, 1, dtype=int)),
    'subsample' : hp.quniform('subsample', 0.5, 1, 0.05),
    'colsample_bytree' : hp.quniform('colsample_bytree', 0.5, 1, 0.05), 
    'objective' : 'reg:squarederror',
    'n_estimators' :  100,    
    'seed' : 0
}

best = fmin(obj_func, xgb_reg_params, algo=tpe.suggest, max_evals=25)

best

Training with params: 
{'colsample_bytree': 0.75, 'learning_rate': 0.3, 'max_depth': 11, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.8500000000000001}
Training with params: 
{'colsample_bytree': 0.7000000000000001, 'learning_rate': 0.25, 'max_depth': 15, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.55}
Training with params: 
{'colsample_bytree': 0.55, 'learning_rate': 0.25, 'max_depth': 12, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.8500000000000001}
Training with params: 
{'colsample_bytree': 0.9, 'learning_rate': 0.05, 'max_depth': 10, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.7000000000000001}
Training with params: 
{'colsample_bytree': 0.55, 'learning_rate': 0.3, 'max_depth': 9, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.5}
Training with params: 
{'colsample_bytree': 0.55, 'learning_rate': 0.2, 'max_de

{'colsample_bytree': 0.9500000000000001,
 'learning_rate': 1,
 'max_depth': 9,
 'subsample': 0.9500000000000001}